In [ ]:
!pip install halo pyarrow numerapi lightgbm optuna

import sys
import os
import pickle
from halo import Halo
import json
import gc
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
import numerapi
import lightgbm
import matplotlib
import optuna
from tqdm import tqdm

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
sys.path.insert(0, './utils')

from utils import (
    save_model,
    load_model,
    neutralize,
    validation_metrics,
    ERA_COL,
    DATA_TYPE_COL,
    TARGET_COL,
    EXAMPLE_PREDS_COL
)
from metrics import evaluate, smart_sharpe, max_feature_exposure
from cross_validation import PurgedTimeSeriesSplitGroups, PurgedKFold

from numerapi import NumerAPI
napi = NumerAPI()

current_round = napi.get_current_round(tournament=8)  # tournament 8 is the primary Numerai Tournament

train_pq_path = "./numeraidata/numerai_training_data.parquet"

# read in just those features along with era and target columns
with open("./numeraidata/features.json") as f:
    feature_metadata = json.load(f)

features = list(feature_metadata["feature_sets"]["medium"])

read_columns = features + [DATA_TYPE_COL, ERA_COL, TARGET_COL]
df_train = pd.read_parquet(train_pq_path, columns=read_columns)

eras = df_train.era.astype(int)
df_train["era"] = eras

gc.collect()

  Preparing metadata (setup.py) ... -

 done


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/26.7 MB ? eta -:--:--

     ━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/26.7 MB 24.1 MB/s eta 0:00:02

     ━━━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/26.7 MB 24.2 MB/s eta 0:00:02

...

 \

 done
  Created wheel for halo: filename=halo-0.0.31-py3-none-any.whl size=11261 sha256=7961199119659d5dd07294cedc31b26447837ed813a4350e4f46bba0de8fe244
  Stored in directory: /tmp/pip-ephem-wheel-cache-ezmshnij/wheels/bb/85/47/b7c7338ab52808105f937bd8c04aec5d98a543311ac2c8bed2


 done
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4848 sha256=475352adcde581d5cc349783901d52a6e11e5f088dac17efc691ecff956fc219
  Stored in directory: /tmp/pip-ephem-wheel-cache-ezmshnij/wheels/b6/0d/90/0d1bbd99855f99cb2f6c2e5ff96f8023fad8ec367695f7d72d


 done
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=2d0167e3a2314bd51a1841cc3e2c7f19bb781c8e2213425000e0f108cee82d8c
  Stored in directory: /tmp/pip-ephem-wheel-cache-ezmshnij/wheels/0c/09/9e/49e21a6840ef7955b06d47394afef0058f0378c0914e48b8b8
Successfully built halo termcolor pyperclip


0

In [ ]:
def objective(trial, X, y):
    
    param_grid = {
        'n_estimators': 5, 
        'learning_rate': 0.01, 
        'num_leaves': 25, 
        'max_depth': 6, 
        'colsample_bytree': 0.1,
        "max_bin": 50,
    }
    
    exps = []
    sharpes = []

    n_splits = 4
    purge = 5
    split = 0

    cv=PurgedTimeSeriesSplitGroups(n_splits, purge)
    
    # get out of sample training preds via embargoed time series cross validation
    print(f"entering cross validation loop")
    for train_index, test_index in (cv.split(df_train[features], df_train[TARGET_COL], eras)):
        split +=1
            
        gc.collect()
        print(f"doing fold {split} out of {n_splits}")
        
        train_eras = df_train.iloc[train_index, :].era.unique()
        test_eras = df_train.iloc[test_index, :].era.unique()
        print(train_eras)
        print(test_eras)
        # train_split_index = df_train[ERA_COL].isin(train_eras)
        # test_split_index = df_train[ERA_COL].isin(test_eras)

        # model = lightgbm.LGBMRegressor(**param_grid)
        # model.fit(df_train.loc[train_split_index, features],
        #       df_train.loc[train_split_index, [TARGET_COL]])

        # df_train.loc[test_split_index, "prediction"] = model.predict(df_train.loc[test_split_index, features])
        
        # feat_exp = max_feature_exposure(df_train.loc[test_split_index, :], "prediction")
        # smart_s = smart_sharpe(df_train.loc[test_split_index, :], "prediction")
        
        # print(f"score: feature exposure: {feat_exp}, smart sharpe: {smart_s}")
        # exps.append(feat_exp)
        # sharpes.append(smart_s)
        
    return np.mean(exps), np.mean(sharpes)

In [ ]:
import optuna

sampler = optuna.samplers.TPESampler() 
study = optuna.create_study(directions=["minimize", "maximize"], study_name="medium_feature_search", sampler=sampler)
func = lambda trial: objective(trial, df_train[features], df_train[TARGET_COL])
study.optimize(func, n_trials=20, timeout=15500, show_progress_bar=True)

[I 2022-03-11 16:51:25,808]

 A new study created in memory with name: medium_feature_search

/opt/conda/lib/python3.9/site-packages/optuna/progress_bar.py:47: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/20 [00:00<?, ?it/s]

entering cross validation loop

...

doing fold 3 out of 4

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162
 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180
 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216
 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234
 235 236 237 238 239 240 241 242 243 244 245 246 24

[349 350 351 352 353 354 355 356 357 358 359 360 361 362 363 364 365 366
 367 368 369 370 371 372 373 374 375 376 377 378 379 380 381 382 383 384
 385 386 387 388 389 390 391 392 393 394 395 396 397 398 399 400 401 402
 403 404 405 406 407 408 409 410 411 412 413 414 415 416 417 418 419 420
 421 422 423 424 425 426 427 428 429 430 431 432 433 434 435 436 437 438
 439 440 441 442 443 444 445 446 447 448 449 450 451 452 453 454 455 456
 457 458 459 460 461]

doing fold 4 out of 4

KeyboardInterrupt: 